# RayTracing Julia - A Julia Software Package for Machine Learning Based Ray Tracing

## 2D Model with Homogenous Velocity

Thoughout this notebook the processing procedures required to generated a ray-tracing procedure using machine learning

In [20]:
using Pkg
#Pkg.add("Flux")
#Pkg.add("Plots")
#Pkg.add("Zygote")

using Flux
using Zygote
using Plots
using LinearAlgebra

Constructing some input data where $X$ represents the source reciever locations in form $[x_s,y_s,x_r,y_r]$ and $Y$ is the velocity at the reciever location $[Y_r]$

In [56]:
# ----- Defining the dataset ------
numsamples = 100

x = rand(Float32,4,numsamples);
y = ones(Float32,numsamples).*0.5;
data = [(x,y)];

Defining the network achietexture, see paper for more info

In [57]:
struct IdentitySkip
   inner
   activation
end

(m::IdentitySkip)(x) = m.inner(x) .+ x

fc0      = Dense(2*2,32,relu)
fc1      = Dense(32,512,relu)
rn1      = IdentitySkip(Dense(512, 512), relu)
rn2      = IdentitySkip(Dense(512, 512), relu)
rn3      = IdentitySkip(Dense(512, 512), relu)
rn4      = IdentitySkip(Dense(512, 512), relu)
rn5      = IdentitySkip(Dense(512, 512), relu)
rn6      = IdentitySkip(Dense(512, 512), relu)
rn7      = IdentitySkip(Dense(512, 512), relu)
rn8      = IdentitySkip(Dense(512, 512), relu)
rn9      = IdentitySkip(Dense(512, 512), relu)
rn10     = IdentitySkip(Dense(512, 512), relu)
fc8      = Dense(512,32,relu)
fc9      = Dense(32,1,abs)
m = Flux.Chain(fc0,fc1,rn1,rn2,rn3,rn4,rn5,rn6,rn7,rn8,rn9,rn10,fc8,fc9);

In [107]:
function loss(x,y)
    # Determining information about the output
    tau  = m(x)
    dtau(x) = x -> gradient(tau,x)[1]
    display(gradient(()->m(x),m))
    
    tau   = reshape(tau,:,1)
    T0    = .√((x[3,:] .- x[1,]).^2 + (x[4,:] .- x[2,:]).^2)
    T1    = (T0.^2).*(dtau[3,:].^2 .+ dtau[4,:].^2)
    T2    = (dtau[3,:].*(x[3,:] .- x[1,:]) .+ dtau[4,:].*(x[4,:] .- x[2,:])).* 2. .* tau
    T3    = tau.^2
    Slo   = (T1.+T2.+T3)
    Vel   = (1 ./Slo)
    TT    = T0.*tau     

    Vel = reshape(Vel,1,:)
    y   = reshape(y,1,:)
    
    # Determining the mean squared error 
    Flux.mse(Vel,y)

end


# ---- Defining the Parameters and optimiser
ps  = Flux.params(m);
opt = ADAM(0.00001)

gs = gradient(() -> loss(x, y), params(m))


MethodError: MethodError: no method matching (::var"#190#193"{Array{Float32,2}})(::Chain{Tuple{Dense{typeof(relu),Array{Float32,2},Array{Float32,1}},Dense{typeof(relu),Array{Float32,2},Array{Float32,1}},IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,IdentitySkip,Dense{typeof(relu),Array{Float32,2},Array{Float32,1}},Dense{typeof(abs),Array{Float32,2},Array{Float32,1}}}})
Closest candidates are:
  #190() at In[107]:5

In [87]:
Flux.train!(loss,ps,data,opt,cb = () -> println("training"))

1.976895579107475

ErrorException: Output should be scalar; gradients are not defined for output nothing